## TODO
Différence avec le pattern Decorator
Decorateurs implémentables à l'aide de classes (on est alors très proche du pattern), peut rendre la mécanique plus lisible: https://python-3-patterns-idioms-test.readthedocs.io/en/latest/PythonDecorators.html#decorators-vs-the-decorator-pattern

# Décorateurs Python 

Ce notebook a été principalement construit à partir de ce [post](http://sametmax.com/comprendre-les-decorateurs-python-pas-a-pas-partie-1/) et de ce [post](http://sametmax.com/comprendre-les-decorateurs-python-pas-a-pas-partie-2/).

En Python les fonctions sont des objets, on peut donc : 
* Les assigner à une variable
* En définir à l'intérieur d'une autre fonction, avoir une fonction qui retourne une fonction, passer une fonction en argument à une autre fonction.

**Remarque :** distinguer : 
* ```myFunc``` qui fait référence à l'objet
* ```myFunc()``` qui appelle la fonction ```myFunc``` si celle-ci ne prend pas ou peut ne prendre aucun argument.

Un décorateur est simplement une fonction qui prend en argument une autre fonction f et qui retourne une nouvelle fonction. Cela permet par exemple d'étendre les fonctionnalités de f dans le cas où on ne pourrait pas la modifier.

Un décorateur va en fait retourner un wrapper contruit autour de la fonction passée en arguments. A l'appel de la fonction passée au décorateur on va ajouter du code avant et/ou après l'appel de cette fonction.

In [10]:
def myDecorator(f):
    def wrapper():
        print "Code placé avant l'appel à la fonction décorée"
        f()
        print "Code placé après l'appel à la fonction décorée"
    return wrapper

def myFunc():
    print "Je suis une fonction intouchable"
    
# Décorer myFunc avec myDecorator correspond exactement à l'opération :
myFunc = myDecorator(myFunc)
# On replace en fait myFunc par wrapper. Appeler myFunc correspondra à appeler la fonction wrapper

In [11]:
myFunc() 

Code placé avant l'appel à la fonction décorée
Je suis une fonction intouchable
Code placé après l'appel à la fonction décorée


En syntaxe Python cette opération s'écrit : 

In [13]:
@myDecorator
def myOtherFunc():
    print "Je suis une autre fonction intouchable"
    
myOtherFunc()

Code placé avant l'appel à la fonction décorée
Je suis une autre fonction intouchable
Code placé après l'appel à la fonction décorée


**Remarque :** le décorateur n'est appelé qu'une fois au chargement de la fonction. Une fois chargée, la fonction non décorée n'est plus accessible car la décoration revient à écraser l'ancienne fonction (```myFunc = myDecorator(myFunc)```)

Rien ne nous empêche de modifier davantage le comportement de la fonction décorée : 

In [16]:
def print2f(f):
    def wrapper(arg1):
        print "Code placé avant l'appel à la fonction décorée"
        print 2*f(arg1) # Ou n'importe quelle autre fonctionelle de f
        print "Code placé après l'appel à la fonction décorée"
    return wrapper

# Attention : ici wrapper et myFunc n'ont pas le même type de retour. Un décorateur n'étant que la composée de fonction, il est
# techniquement possible que le type de retour de la fonction décorée ne soit pas le même que celui de la fonction non décorée.
# Il est possible que cela ne soit pas une bonne pratique : il n'est pas possible de savoir facilement, au premier coup d'oeil
# quel sera le type retourné par la fonction décorée.

@print2f
def myFunc(arg1):
    return arg1+1

myFunc(2)

Code placé avant l'appel à la fonction décorée
6
Code placé après l'appel à la fonction décorée


On remarque qu'il faut faire un peu plus d'efforts quand la fonction à décorer prend des arguments. Comme décorer la fonction revient en fait à appeler la fonction ```wrapper```, il faut que celle-ci prenne les mêmes arguments que la fonction décorée. 

Cela peut nous permettre au passage d'ajouter des comportements utilisant ces arguments.

In [21]:
def myDecorator(f):
    def wrapper(arg1, arg2):
        print "Code placé avant l'appel à la fonction décorée"
        print "Premier argument : " + str(arg1)
        print "Second argument : " + str(arg2)
        print f(arg1, arg2) # Ou n'importe quelle autre fonctionelle de f
        print "Code placé après l'appel à la fonction décorée"
    return wrapper

@myDecorator
def myFunc(arg1, arg2):
    return arg1+arg2

myFunc(1, 2)

Code placé avant l'appel à la fonction décorée
Premier argument : 1
Second argument : 2
3
Code placé après l'appel à la fonction décorée


En programmation objet, on peut ainsi modifier le comportement d'une méthode avec un décorateur du type :

```python
def myDecorator(myMethod):
    def wrapper(self):
        ...
    return wrapper
```

Pour être paré à tout on peut faire en sorte que le décorateur accepte n'importe quels arguments en utilisant ```*args``` et ```**kwargs``` : 

```python
def myDecorator(myMethod):
    def wrapper(*args, **kwargs):
        ...
    return wrapper
```

Toutefois cela pose des problèmes du point de vue de l'introspection (qui rend entre autre possible des choses comme l'autocomplétion. En effet comme la décoration a pour effet d'écraser la fonction décorée avec son wrapper, on perd du même coup toutes les "méta-informations" de cette fonction (docstring, etc.). Pour palier à cela, le module ```functools``` fournit le décorateur ```@wraps``` qui copie littéralement toutes les infos d'une fonction sur son wrapper. Il suffit en fait de décorer le wrapper avec ```@wraps```

```python
def myDecorator(myMethod):
    @wraps(myFunc)
    def wrapper(arg1, arg2):
        ...
        myFunc(arg1, arg2)
        ...
    return wrapper
```

**Remarque :** Si on accepte ```*args``` et ```**kwargs```, la liste des arguments ne sera plus disponible à l'introspection sans que ```@wraps``` puisse y faire quoi que ce soit.

**Remarque : Introspection (et reflection) en Python :** 

On peut évidemment chaîner les décorateurs :

In [22]:
def pain(func):
    def wrapper():
        print("</''''''\>")
        func()
        print("<\______/>")
    return wrapper
 
def ingredients(func):
    def wrapper():
        print("#tomates#")
        func()
        print("~salade~")
    return wrapper
 
@pain
@ingredients
def sandwich(food="--jambon--"):
    print(food)

sandwich()

</''''''\>
#tomates#
--jambon--
~salade~
<\______/>


L'enchainement :

```python
@pain
@ingredients
def sandwich
```

est équivalent à ```sandwich = pain(ingredients(sandwich))```

**Attention**, il est alors évident que l'ordre importe : 

In [23]:
@ingredients
@pain
def sandwich(food="--jambon--"):
    print(food)

sandwich()

#tomates#
</''''''\>
--jambon--
<\______/>
~salade~


## Décorateurs à argument
On remarque qu'on peut passer un argument au décorateur ```@wraps```. Il existe donc un moyen de paramétrer l'utilisation d'un décorateur. Créer un décorateur "paramétrable" n'est pas aussi simple. La fonction prenant le paramètre en argument ne retourne pas de wrapper mais un décorateur (qui lui retourne le wrapper d'une fonction à décorer). 
Ainsi quand on décore ```wrapper``` avec ```@wraps(myFunc)```, on décore en fait avec le décorateur retourné par ```wraps(myFunc)```.  

In [38]:
def customizableDecorator(arg1dec):
 
    print("Je fabrique des décorateurs. Je suis éxécuté une seule fois à la création du décorateur. "
         + "En tant que créateur de décorateur, je retourne un décorateur.\n")
 
    def myDecorator(f):
 
        print("Je suis un décorateur, je suis éxécuté une seule fois quand on décore la fonction. "
              + "En tant que décorateur, je retourne le wrapper.\n")
 
        def wrapper(arg1f, arg2f):
            print("Je suis le wrapper autour de la fonction décorée.\n"
                  "Je suis appelé quand on appelle la fonction décorée.\n"
                  "En tant que wrapper, je retourne le RESULTAT de la fonction décorée.\n")
            result = f(arg1f, arg2f)**arg1dec
            print "Résultat de l'appel de " + f.__name__ + " : " + str(result)
            return result
        return wrapper    
    return myDecorator

@customizableDecorator(2) # Création du décorateur et décoration de la fonction on en fait lieu en même temps.
def myFunc1(arg1, arg2):
    return arg1+arg2

@customizableDecorator(3)
def myFunc2(arg1, arg2):
    return arg1+arg2

Je fabrique des décorateurs. Je suis éxécuté une seule fois à la création du décorateur. En tant que créateur de décorateur, je retourne un décorateur.

Je suis un décorateur, je suis éxécuté une seule fois quand on décore la fonction. En tant que décorateur, je retourne le wrapper.

Je fabrique des décorateurs. Je suis éxécuté une seule fois à la création du décorateur. En tant que créateur de décorateur, je retourne un décorateur.

Je suis un décorateur, je suis éxécuté une seule fois quand on décore la fonction. En tant que décorateur, je retourne le wrapper.



In [39]:
myFunc1(1, 2)
print "-------------------------"
myFunc2(1, 2)

Je suis le wrapper autour de la fonction décorée.
Je suis appelé quand on appelle la fonction décorée.
En tant que wrapper, je retourne le RESULTAT de la fonction décorée.

Résultat de l'appel de myFunc1 : 9
-------------------------
Je suis le wrapper autour de la fonction décorée.
Je suis appelé quand on appelle la fonction décorée.
En tant que wrapper, je retourne le RESULTAT de la fonction décorée.

Résultat de l'appel de myFunc2 : 27


27

**Remarque :** comme précisé plus haut, le décorateur n'est appelé qu'une seule fois à la définition, au chargement de la fonction décorée. On a donc pas à le réécrire à chaque appel de la fonction (puisque la fonction décorée écrase la fonction non décorée).

## A quoi sert un décorateur ?
Potentiellement à plein de choses puisqu'il permet de modifier, d'étendre le comportement d'une fonction (qu'on peut ne pas pouvoir modifier comme pour des fonctions de modules externes). On peut entre autres ainsi ajouter des fonctionnalités à la fontion, gérer les permissions d'une fonction, réagir aux arguments passés à la fonction (ses arguments étant passés au wrapper), débugger, etc. 

Exemples de décorateurs classiques :

```python
def timer(func):
    """
    Un décorateur qui affiche le temps qu'une fonction met à s'éxécuter
    """
    import time
    def wrapper(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        print(func.__name__, time.clock()-t)
        return res 
    return wrapper
 
def logging(func):
    """
    Un décorateur qui log l'activité d'un script.
    (Ca fait un print, mais ça pourrait logger !)
    """
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(func.__name__, args, kwargs)
        return res
    return wrapper
 
def counter(func):
    """
    Compte et affiche le nombre de fois qu'une fonction a été exécutée
    """
    def wrapper(*args, **kwargs):
        wrapper.count = wrapper.count + 1
        res = func(*args, **kwargs)
        print("{0} a été utilisée: {1}x".format(func.__name__, wrapper.count))
        return res
    wrapper.count = 0
    return wrapper
```

On remarque que les types de sortie des fonctions décorées ne sont pas (et n'ont pas forcément à être) les mêmes que ceux des fonctions non décorées.

Prolongement: 
https://www.codementor.io/sheena/advanced-use-python-decorators-class-function-du107nxsv